In [1]:
import numpy as np

<div class="alert alert-info"><center><h1>Биологический нейрон</h1></center></div>

**Нейрон** — электрически возбудимая клетка, которая предназначена для приёма извне, обработки, хранения, передачи и вывода вовне информации с помощью электрических и химических сигналов.  
  
Типичный нейрон состоит из тела клетки, дендритов и одного аксона:

![neuron](images/neuron.png)

Простыми словами можно описать принцип действия нейрона следующим образом:
- через дендриты в нейрон поступают сигналы (раздражители)
- Если комбинация сигналов превышает пороговый уровень - нейрон "выстреливает", т.е. передаёт сигнал дальше через аксон.

Нейроны могут соединяться один с другим, формируя нервные сети.

<div class="alert alert-info"><center><h1>Функции Активации (Activation Functions)</h1></center></div>

Простейшим механизмом активации является **Step activation**, когда перспептрон передаёт на выход значение только в том случае, если сумма взвешенных входящих сигналов больше заданного порога:

![step](images/step_activation.png)

При всей своей простоте, данная функция активации обладает критическим недостатком: она недифференцируемая.  Как результат, она не позволяет осуществлять процесс обучения персептрона.  
  
Для того, чтобы исправить это, было разработано множество других функций активаций, таких как:

![neuron](images/activation_functions.png)

<div class="alert alert-warning"><center><h1>Задание 1 (2 балла)</h1></center></div>

Напишите класс **ActivationFunction** и его подкласс **Sigmoid**, у которого будет функция `forward`, которая:
- будет принимать на вход число и будет сохранять его внутри объекта
- будет возвращать результат в соответствии с фукцией $\sigma(x) = \frac{1}{1 + e^{-x}}$

In [3]:
class ActivationFunction:
    
    def __init__(self) -> None:
        self.input = None
        self.output = None
    
    def forward(self) -> None:
        return 


class Sigmoid(ActivationFunction):

    def forward(self, value: float) -> float:
        self.input = value
        self.output = 1.0/(1.0+np.exp(-self.input))
        return self.output


In [4]:
sigm = Sigmoid()

In [5]:
sigm.forward(1.5)

0.8175744761936437

In [6]:
sigm.input, sigm.output

(1.5, 0.8175744761936437)

---

---

<div class="alert alert-info"><center><h1>Персептрон</h1></center></div>

**Персептрон** -  математическая модель биологического нейрона, является базовым элементом нейронных сетей:

![neuron](images/perceptron.png)

**Персептрон** состоит из следующих ключевых элементов:
- `вход` - отвечает за получение входных значений. Является аналогом дендрита биологического нейрона
- `веса` - механизм "важности" входных значений. По аналогии с нейроном - это "толщина" дендрита
- `функция активации` - обрабатывает сумму взвешенных входных сигналов и передаёт результат на выход
- `выход` - отвечает за передачу итогового результата. Аналогичен аксону
  
Практически всегда к входным сигналам также добавляется "bias", который всегда = 1.  
Это позволяет не привязывать выход персептрона к 0 в случае, если все входные сигналы также равны 0 (как в механизме регрессии)

<div class="alert alert-warning"><center><h1>Задание 2 (4 балла)</h1></center></div>

напишите класс **Layer**, у когорого будут следующие входные параметры:
- **n_inputs** - количество входящих значений
- **n_outputs** - количество исходящих значений (в нашем случае = 1)
- **activation** - объект из семейства **ActivationFunction** (в нашем случае - **Sigmoid**)
  
При своём создании объект класса **Layer** должен также создавать атрибут `weights_`, в ктором будут рандомально инициализированны веса для входящих значений, а также для `bias`

Класс **Layer** должен иметь функцию `forward`, принимающую на вход массив *numpy*, и возвращающую результат функции активации (тоже в виде массива).  
Также эта функция должна сохранять полученные на вход значения внутри экземпляра

In [33]:
class Layer:

    def __init__(self, n_inputs=1, n_outputs=1, activation=Sigmoid()):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.input = None
        self.z = None
        self.output = None
        self.activation = activation
        self.bias = np.random.normal()
        self.weights_ = np.random.normal(size=self.n_inputs+1)

    def forward(self, X: np.ndarray) -> np.ndarray:
        assert X.shape[0] == self.n_inputs, f'Размер входа не равен {self.n_inputs}'

        self.input = X
        self.z = np.hstack((self.input, np.array(self.bias))).dot(self.weights_.T)
        self.output = self.activation.forward(self.z)

        return self.output



In [40]:
perceptron = Layer(5)

In [41]:
perceptron.forward(np.array([1,2,3,4,5]))

0.6359993901142399

In [42]:
perceptron.weights_

array([-0.45887725,  0.18555862,  0.52939684,  0.48375538, -0.52247824,
       -0.86594063])

In [43]:
perceptron.bias

0.30604725538882693

---

<div class="alert alert-warning"><center><h1>Задание 3 (2 балла)</h1></center></div>

напишите класс **LossFunction** и его подкласс **CrossEntropy**, у которого будет функция `loss`, которая будет принимать реальное бинарное значение *y_fact* и вероятность *y_prob* (оба параметра в виде np.array) и будет возвращать результат по формуле:  
  
$$
L = - \sum (y_{fact} * log(y_{prob}) + (1-y_{fact})*log(1-y_{prob}))
$$

In [ ]:
class LossFunction:
    ...

class CrossEntropy (LossFunction):

    def loss(self, y_fact: np.ndarray, y_prob: np.ndarray):
        ...

---

<div class="alert alert-info"><center><h1>Обучение. Forward and Backpropagation</h1></center></div>

Процесс обучения персептрона (и в целом нейросети) итеративен и состоит из следующих этапов:
- Сперва персептрон инициализируется с рандомальными весами
- Осуществляется цикл "вперёд":
  - Входные значения перемножаются с соответствующими весами и суммируются
  - Эта сумма подаётся на функцию активации
  - Функция активации возвращает итоговое значение
- Итоговое значение сравнивается с ожидаемым и высчитывается ошибка (Loss)
- Осуществляется цикл "назад":
  - при помощи `Chain Rule` рассчитываются частичные производные для всех элементов персептрона
  - исходя из заданного коэффициента обучения (`learning rate`, $\alpha$), веса $w_{i}$ корректируются
- Данный цикл повторяется заданное количество раз или до тех пор, пока итоговая ошибка не опустится ниже заданного порогового значения

![img](images/training.png)

### <center>Chain Rule</center>

Если нам дана функция $y=f(u)$, где $u = g(x)$, то тогда производная этой функции по $x$ будет равно:

$$
\frac{dy}{dx} = \frac{dy}{du}\frac{du}{dx}
$$

Тогда для того, чтобы понять, насколько изменение весов $w$ влияет на изменение $y$ (т.е. производные $\frac{dy}{dw_{i}}$), можно вычислить следующие производные:

![neuron](images/backpropagation.png)

<div class="alert alert-warning"><center><h1>Задание 4 (8 баллов)</h1></center></div>

Модифицируйте классы **Layer**, **LossFuncton** и **ActivationFunction** таким образом, чтобы можно было рассчитать их частичные производные, и добавьте функцию `back`, позволяющую осуществить backpropagation.

<div class="alert alert-danger"><center><h3>Это задание очень сложное, и даже частичное его выполнение будет учитываться</h3></center></div>

---

# <center>Удачи!</center>